In [1]:
using Gmsh
using Gridap
using GridapGmsh
using GridapMakie
using GLMakie
using SparseArrays
using ChainRulesCore
using Zygote
using LinearAlgebra
using KrylovKit
using PartitionedArrays
using NLopt
using DelimitedFiles
using Gridap.Geometry
using Gridap.Fields

import ChainRulesCore: rrule
import Gmsh: gmsh

include("Mesh_Periodic.jl")
include("Mesh_CR.jl")
include("Mesh_RR.jl")
include("Control.jl")
include("Helper.jl")
include("GridapFE.jl")
include("Model.jl")

VectorO (generic function with 1 method)

In [2]:
include("DefaultParameters.jl")

# Change default geometry parameters
rd = 0.5
rt = rd + 0.2

# Create mesh file
geo_param = CirRecGeometry(L, H, rd, rt, dpml, l1, l2, l3)
#geo_param = PeriodicGeometry(L, h1, h2, h3, ht, hs, dpml, l1, l2, l3)
meshfile_name = "geometry.msh"
MeshGenerator(geo_param, meshfile_name)

# Apply gridap finite element analysis to mesh file
gridap = GridapFE(meshfile_name, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], [], flag_f)
#run(`gmsh geometry.msh`)

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Meshing curve 2 (Line)
Info    : Meshing curve 3 (Line)
Info    : Meshing curve 4 (Line)
Info    : Meshing curve 5 (Line)
Info    : Meshing curve 6 (Line)
Info    : Meshing curve 7 (Line)
Info    : Meshing curve 8 (Line)
Info    : Meshing curve 9 (Line)
Info    : Meshing curve 10 (Line)
Info    : Meshing curve 11 (Line)
Info    : Meshing curve 12 (Line)
Info    : Meshing curve 13 (Line)
Info    : Meshing curve 14 (Line)
Info    : Meshing curve 15 (Line)
Info    : Meshing curve 16 (Line)
Info    : Meshing curve 17 (Line)
Info    : Meshing curve 18 (Line)
Info    : Meshing curve 19 (Line)
Info    : Meshing curve 20 (Line)
Info    : Meshing curve 21 (Line)
Info    : Meshing curve 22 (Line)
Info    : Meshing curve 23 (Line)
Info    : Meshing curve 24 (Line)
Info    : Meshing curve 25 (Circle)
Info    : Meshing curve 26 (Circle)
Info    :

GridapParameters(UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), 4650, UnstructuredGrid(), Measure(), Measure(), Measure(), Measure[Measure()], Measure(), Gridap.CellData.GenericCellField[GenericCellField()], Int8[6, 6, 6, 6, 6, 6, 6, 6, 6, 6  …  7, 7, 7, 7, 7, 7, 7, 7, 7, 7], 7)

In [3]:
# Change default physics parameters
kb = VectorValue(2*π*0.,0)
LHp=[L / 2, H / 2]  # Start of PML for x,y > 0
LHn=[L / 2, H / 2]  # Start of PML for x,y < 0

phys = PhysicalParameters(k, kb, ω, ϵ1, ϵ2, ϵ3, ϵd, μ, R, σs, dpml, LHp, LHn, wg_center, wg_size)

# Change default control parameters
Bρ = true
ρv = 0.5

control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bρ, ρv, c, ηe, ηd)

ControllingParameters(true, true, [0.02, 0.02], 40.0, 0.5, 0.0005, 15, 30, 20, 1.0, true, 0.5, 0.0, 0.75, 0.25)

In [9]:
include("ObjectiveDraft.jl")

g_ρ (generic function with 2 methods)

In [8]:
flag_f = true
flag_t = true
Bρ = true
α = 2 / 1000
#gridap = GridapFE(meshfile_name, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], [], flag_f)
control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bρ, ρv, c, ηe, ηd)

ρ0 = rand(gridap.np) * 1
δρ = rand(gridap.np) * 1e-6
ρf_vec = ρf_ρ0(ρ0; control, gridap)
ρfh = FEFunction(gridap.FE_Pf, ρf_vec)
ρth = (ρh -> Threshold(ρh; control)) ∘ ρfh

N = num_free_dofs(gridap.FE_U)
W_mat = rand(ComplexF64, N, control.K)
B_mat = MatrixB(ρth; control, gridap)
O_mat = MatrixOc(phys.ω, phys.ϵ1; gridap)

grad = zeros(gridap.np)
g1 = g_ρ(ρ0 + δρ; O_mat, B_mat, W_mat, phys, control, gridap)
g0 = g_ρ(ρ0, grad; O_mat, B_mat, W_mat, phys, control, gridap)
g1 - g0, grad' * δρ

(2.088285343780638e-7, 3.256967549382752e-7)

In [ ]:
sum(∫(ρth)gridap.dΩ_d)

In [ ]:
ρ0 = rand(gridap.np) * 1
ρf_vec = ρf_ρ0(ρ0; control, gridap)
ρfh = FEFunction(gridap.FE_Pf, ρf_vec)
ρth = (ρh -> Threshold(ρh; control)) ∘ ρfh

N = num_free_dofs(gridap.FE_U)
W_mat = rand(ComplexF64, N, control.K)
A_mat = MatrixA(ρth; phys, control, gridap)
B_mat = MatrixB(ρth; control, gridap)
O_mat = MatrixOc(phys.ω, phys.ϵ1; gridap)

G_ii, W_raw, info = eigsolve(x -> MatrixG(x; A_mat, B_mat, O_mat), rand(ComplexF64,N), control.K, :LM; krylovdim=30)
for ib = 1 : control.K
    W_mat[:, ib] = W_raw[ib]
end
WBW = W_mat' * B_mat * W_mat
U_mat = U_ρf(ρf_vec; B_mat, W_mat, phys, control, gridap)
sum(real.(G_ii)), g_U(U_mat; O_mat, WBW)

In [ ]:
PowerMatrixTemp = A_mat \ (A_mat\ Matrix(B_mat))
PowerVector = diag(PowerMatrixTemp)
Ih = FEFunction(gridap.FE_U, PowerVector)

In [ ]:
fig, ax, plt = plot(gridap.Ω, real(Ih), colormap=:heat)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/H)
limits!(ax, -L/2, L/2, -phys.LHn[2], phys.LHp[2])
fig

In [ ]:

#btest(v) = ∫(v*(x -> GaussianD(x, [0.0, 0.0], [l1, l1])))gridap.dΩ
btest(v) = ∫(v)gridap.dΓ_s
b_vec = assemble_vector(btest, gridap.FE_V)
A_mat = MatrixA(ρth; phys, control, gridap)
#B_mat = MatrixB(ρth; control, gridap)

u_vec = A_mat\b_vec
uh = FEFunction(gridap.FE_U, u_vec)*(x->cis(phys.kb⋅x))

fig, ax, plt = plot(gridap.Ω, real(uh), colormap=:bwr)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(H))
limits!(ax, -L/2, L/2, -phys.LHn[2], phys.LHp[2])
fig

In [ ]:
btest(v) = ∫(v*(x -> GaussianD(x, [-0.1, -0.5], [l1, l1])))gridap.dΩ
b_vec = assemble_vector(btest, gridap.FE_V)
u_vec = A_mat\b_vec
uh = FEFunction(gridap.FE_U, u_vec)

fig, ax, plt = plot(gridap.Ω, real(uh), colormap=:bwr)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(h1+h2+h3))
limits!(ax, -L/2, L/2, -phys.LHn[2], phys.LHp[2])
fig